In [1]:
import pandas as pd
import matplotlib.pyplot as pl
from gensim.models import Word2Vec,KeyedVectors
import nltk
from nltk.corpus import stopwords
import re

from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.preprocessing import LabelEncoder


from sklearn.metrics.pairwise import cosine_similarity,cosine_distances

In [2]:
df=pd.read_excel('data.xlsx')

In [3]:
df.head()

,Category,Tag,Question,Answer
0,My Account,Rating and Reviews,How to Review Product as well as Seller?,Everything is right here. Please check this ou...
1,My Account,Rating and Reviews,How to Filter Search Results with Product Rati...,Products can also be filtered according to the...
2,My Account,Rating and Reviews,What are Ratings & Reviews and how do I write ...,Daraz customers have the option of rating and ...
3,My Account,Product Warranty,How do I know if a product comes with free ins...,Unfortunately we are not providing the Free In...
4,My Account,Product Warranty,How do I know if a product comes with warranty?,If a warranty is offered on a product the warr...


In [4]:
txt=df['Answer'][2]
txt

'Daraz customers have the option of rating and reviewing their product and seller experiences.,The customers can both leave star ratings and add text to describe what they liked or disliked about their experience.'

In [5]:
txt.split(',')

['Daraz customers have the option of rating and reviewing their product and seller experiences.',
 'The customers can both leave star ratings and add text to describe what they liked or disliked about their experience.']

In [6]:
df['Question'].head()

0             How to Review Product as well as Seller?
1    How to Filter Search Results with Product Rati...
2    What are Ratings & Reviews and how do I write ...
3    How do I know if a product comes with free ins...
4      How do I know if a product comes with warranty?
Name: Question, dtype: object

In [7]:
def cleantext(txt):
    txt=txt.split()
    txt=[i.lower() for i in txt if i not in stopwords.words('english') ]
    txt=' '.join(txt)
    txt=re.sub(r'[^A-Za-z0-9]',' ',txt)
    txt=' '.join(txt.split())
    return txt 
    
    
cleantext('My name  is  rishav..') 
df['Cleaned']=df['Question'].apply(lambda x :cleantext(x))

In [8]:
df['Cleaned'].head(10)

0                how review product well seller
1     how filter search results product ratings
2     what ratings reviews i write good reviews
3    how i know product comes free installation
4             how i know product comes warranty
5     i lost warranty card how i claim warranty
6        are hidden costs charges i order daraz
7                   are prices daraz negotiable
8            why i see different prices product
9                            how contact vendor
Name: Cleaned, dtype: object

In [9]:
vectorizer=TfidfVectorizer()
X_=vectorizer.fit_transform(df['Cleaned']).toarray()

In [10]:
X_[:2]

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.28192401, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.  

In [11]:
X_.shape

(44, 123)

In [12]:
labelencoder=LabelEncoder()
df['Label']=labelencoder.fit_transform(df['Category'])

In [13]:
df.head()

,Category,Tag,Question,Answer,Cleaned,Label
0,My Account,Rating and Reviews,How to Review Product as well as Seller?,Everything is right here. Please check this ou...,how review product well seller,1
1,My Account,Rating and Reviews,How to Filter Search Results with Product Rati...,Products can also be filtered according to the...,how filter search results product ratings,1
2,My Account,Rating and Reviews,What are Ratings & Reviews and how do I write ...,Daraz customers have the option of rating and ...,what ratings reviews i write good reviews,1
3,My Account,Product Warranty,How do I know if a product comes with free ins...,Unfortunately we are not providing the Free In...,how i know product comes free installation,1
4,My Account,Product Warranty,How do I know if a product comes with warranty?,If a warranty is offered on a product the warr...,how i know product comes warranty,1


In [14]:
from sklearn.linear_model import LogisticRegression
logistic=LogisticRegression()

In [15]:
X=X_.copy()
y=df['Label']

In [16]:
logistic.fit(X,y)

LogisticRegression()

In [17]:
logistic.score(X,y)

0.9090909090909091

In [18]:
def return_tfidf_data(text):
    return vectorizer.transform([text])

def return_similarity(text):
    text=cleantext(text)
    print(text)
    text=return_tfidf_data(text)
    return text.toarray()
    
    
temp=return_similarity('How to check warrenty of product?')
temp

how check warrenty product


array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.75885275, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.41354658, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.  

In [20]:
# cos_sim=cosine_similarity(A.reshape(1,-1),B.reshape(1,-1))
# print (f"Cosine Similarity between A and B:{cos_sim}")
# print (f"Cosine Distance between A and B:{1-cos_sim}")

In [21]:
cos_sim=[cosine_similarity(X_[i].reshape(1,-1),temp.reshape(1,-1)) for i in range(len(X_))]
cos_sim=np.array(cos_sim)

cos_sim=cos_sim.flatten()

<IPython.core.display.Javascript object>

In [22]:
df_temp=df.copy()
df_temp['Cos_sim']=cos_sim

In [23]:
df_temp.head()

,Category,Tag,Question,Answer,Cleaned,Label,Cos_sim
0,My Account,Rating and Reviews,How to Review Product as well as Seller?,Everything is right here. Please check this ou...,how review product well seller,1,0.289147
1,My Account,Rating and Reviews,How to Filter Search Results with Product Rati...,Products can also be filtered according to the...,how filter search results product ratings,1,0.262054
2,My Account,Rating and Reviews,What are Ratings & Reviews and how do I write ...,Daraz customers have the option of rating and ...,what ratings reviews i write good reviews,1,0.000000
3,My Account,Product Warranty,How do I know if a product comes with free ins...,Unfortunately we are not providing the Free In...,how i know product comes free installation,1,0.267625
4,My Account,Product Warranty,How do I know if a product comes with warranty?,If a warranty is offered on a product the warr...,how i know product comes warranty,1,0.313713


In [24]:
df_temp.sort_values('Cos_sim',ascending=False).head()

,Category,Tag,Question,Answer,Cleaned,Label,Cos_sim
32,Ordering,Big mart,Problems with Check Out/ or any issues encount...,If you are facing any error while placing the ...,problems check out issues encountered,2,0.339369
15,DarazMall,What is Darazmall,How can I identify a DarazMall product in the ...,"You can identify"" DarazMall"" products by the ""...",how i identify darazmall product website,0,0.314758
4,My Account,Product Warranty,How do I know if a product comes with warranty?,If a warranty is offered on a product the warr...,how i know product comes warranty,1,0.313713
0,My Account,Rating and Reviews,How to Review Product as well as Seller?,Everything is right here. Please check this ou...,how review product well seller,1,0.289147
3,My Account,Product Warranty,How do I know if a product comes with free ins...,Unfortunately we are not providing the Free In...,how i know product comes free installation,1,0.267625


In [25]:
logistic.predict(temp)

array([1])

In [29]:
df[df['Label']==1].index

Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], dtype='int64')

In [34]:
cos_sim=[cosine_similarity(i.reshape(1,-1),temp.reshape(1,-1)) for i in X_[df[df['Label']==1].index]]
cos_sim=np.array(cos_sim)

cos_sim=cos_sim.flatten()

<IPython.core.display.Javascript object>

In [35]:
cos_sim

array([0.28914749, 0.26205392, 0.        , 0.26762493, 0.31371267,
       0.08807656, 0.        , 0.        , 0.17575454, 0.14870046,
       0.161759  , 0.        ])

In [49]:
temp_df=df.iloc[df[df['Label']==1].index,:]
temp_df['cosine_sim']=cos_sim
temp_df

C:\Users\rissu\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Category,Tag,Question,Answer,Cleaned,Label,cosine_sim
0,My Account,Rating and Reviews,How to Review Product as well as Seller?,Everything is right here. Please check this ou...,how review product well seller,1,0.289147
1,My Account,Rating and Reviews,How to Filter Search Results with Product Rati...,Products can also be filtered according to the...,how filter search results product ratings,1,0.262054
2,My Account,Rating and Reviews,What are Ratings & Reviews and how do I write ...,Daraz customers have the option of rating and ...,what ratings reviews i write good reviews,1,0.000000
3,My Account,Product Warranty,How do I know if a product comes with free ins...,Unfortunately we are not providing the Free In...,how i know product comes free installation,1,0.267625
4,My Account,Product Warranty,How do I know if a product comes with warranty?,If a warranty is offered on a product the warr...,how i know product comes warranty,1,0.313713
5,My Account,Product Warranty,I lost my warranty card. How can I claim warra...,Lost your warranty card? Don't worry! You can ...,i lost warranty card how i claim warranty,1,0.088077
6,My Account,Product Price,Are there any hidden costs or charges if I ord...,There are no hidden costs or charges when you ...,are hidden costs charges i order daraz,1,0.000000
7,My Account,Product Price,Are the prices on Daraz negotiable?,Prices on Daraz are not negotiable. Daraz has ...,are prices daraz negotiable,1,0.000000
8,My Account,Product Price,Why do I see different prices for the same pro...,Daraz is a marketplace. We have a huge seller ...,why i see different prices product,1,0.175755
9,My Account,Product Information,How to contact Vendor?,Now you can easily contact vendor through Dara...,how contact vendor,1,0.148700


In [50]:
temp_df.sort_values('cosine_sim')

,Category,Tag,Question,Answer,Cleaned,Label,cosine_sim
2,My Account,Rating and Reviews,What are Ratings & Reviews and how do I write ...,Daraz customers have the option of rating and ...,what ratings reviews i write good reviews,1,0.000000
6,My Account,Product Price,Are there any hidden costs or charges if I ord...,There are no hidden costs or charges when you ...,are hidden costs charges i order daraz,1,0.000000
7,My Account,Product Price,Are the prices on Daraz negotiable?,Prices on Daraz are not negotiable. Daraz has ...,are prices daraz negotiable,1,0.000000
11,My Account,Product Information,Are all products on Daraz new and unused?,Yes Daraz only offers 100% new and unused prod...,are products daraz new unused,1,0.000000
5,My Account,Product Warranty,I lost my warranty card. How can I claim warra...,Lost your warranty card? Don't worry! You can ...,i lost warranty card how i claim warranty,1,0.088077
9,My Account,Product Information,How to contact Vendor?,Now you can easily contact vendor through Dara...,how contact vendor,1,0.148700
10,My Account,Product Information,Where can I find more detailed information abo...,Information regarding your product are describ...,where i find detailed information product,1,0.161759
8,My Account,Product Price,Why do I see different prices for the same pro...,Daraz is a marketplace. We have a huge seller ...,why i see different prices product,1,0.175755
1,My Account,Rating and Reviews,How to Filter Search Results with Product Rati...,Products can also be filtered according to the...,how filter search results product ratings,1,0.262054
3,My Account,Product Warranty,How do I know if a product comes with free ins...,Unfortunately we are not providing the Free In...,how i know product comes free installation,1,0.267625
